Axioms for context <a class="ProveItLink" href="_context_.ipynb">proveit.physics.quantum.QPE</a>
========

In [ ]:
import proveit
from proveit._common_ import k, A, B, C
from proveit.linalg import MatrixProd, ScalarProd, TensorExp #SU, TensorProd, 
from proveit.logic import Equals, Forall, InSet
from proveit.number import zero, one, two, e, i, pi, Exp, Mult, NaturalsPos, IntervalCO
from proveit.number import Complexes, num # delete later after testing
from proveit.physics.quantum import Ket, RegisterSU, QubitRegisterSpace
# from proveit.physics.quantum._common_ import 
from proveit.physics.quantum.QPE._common_ import n_, psi_k, t_, u_, U_, phase_
# the context is in the current directory:
context = proveit.Context('.') # adds context root to sys.path if necessary

In [ ]:
%begin axioms

Let $U$ be a unitary operator that acts on $n$ qubits, with $\lvert u\rangle$ as an eigenstate of $U$ with eigenvalue $e^{2 \pi i \varphi}$:

In [ ]:
nInNatPos = InSet(n_, NaturalsPos)

In [ ]:
unitaryU = InSet(U_, RegisterSU(n_))

In [ ]:
uKetRegister = InSet(Ket(u_), QubitRegisterSpace(n_))

In [ ]:
phaseInInterval = InSet(phase_, IntervalCO(zero, one))

In [ ]:
Ket(u_)

In [ ]:
MatrixProd(A, B, C)

In [ ]:
MatrixProd(U_, Ket(u_))

In [ ]:
ScalarProd(Exp(e, Mult(two, pi, i, phase_)), Exp(e, Mult(two, pi, i, phase_)), Ket(u_))

In [ ]:
eigenUu = Equals(MatrixProd([U_, Ket(u_)]), ScalarProd(Exp(e, Mult(two, pi, i, phase_)), Ket(u_)))

In [ ]:
tInNaturalsPos = InSet(t_, NaturalsPos)

In [ ]:
psiKetsInQRegSpace = Forall(k, InSet(Ket(psi_k), QubitRegisterSpace(n_)), domain=NaturalsPos)

In [ ]:
%end axioms